# Generate RTX-KG2 Sample Parquet to Kuzu

In [58]:
import gzip
import json
import pathlib
import shutil
from typing import Any, Dict, Generator, List, Literal

import awkward as ak
import ijson
import kuzu
import pyarrow as pa
import requests
from genson import SchemaBuilder
from pyarrow import parquet
from rtx_kg2_functions import (
    count_items_under_top_level_name,
    find_top_level_names,
    parse_items_by_topmost_item_name,
    parse_metadata_by_object_name,
)

In [3]:
# set data to be used throughout notebook
chunk_size = 1
data_dir = "data"
parquet_dir = f"{data_dir}/"
source_data_url = "https://github.com/ncats/translator-lfs-artifacts/raw/main/files/kg2c_lite_2.8.4.json.gz"
target_extracted_sample_data = f"{data_dir}/{pathlib.Path(source_data_url).name.replace('.json.gz', '.sample.json')}"
parquet_dir = target_extracted_sample_data.replace(".json", ".dataset.parquet")
kuzu_dir = target_extracted_sample_data.replace(".json", ".dataset.kuzu")
target_extracted_sample_data_schema_file = target_extracted_sample_data.replace(
    ".json", ".schema.json"
)
print(f"Kuzu dir: {kuzu_dir}")

In [92]:
# create path for the kuzu database to reside
pathlib.Path(kuzu_dir).mkdir(exist_ok=True)

In [93]:
# init a Kuzu database and connection
db = kuzu.Database(f"{kuzu_dir}")
kz_conn = kuzu.Connection(db)

In [113]:
def generate_cypher_table_create_stmt_from_parquet_file(
    parquet_file: str,
    table_type: Literal["node", "rel"],
    table_name: str,
    table_pkey_parquet_field_name: str = "id",
    rel_table_field_mapping: Dict[str, str] = {"from": "Node", "to": "Node"},
):

    parquet_schema = parquet.read_schema(parquet_file)

    if table_pkey_parquet_field_name not in [field.name for field in parquet_schema]:
        raise LookupError(
            f"Unable to find field {table_pkey_parquet_field_name} in parquet file {parquet_file}."
        )

    # Map Parquet data types to Cypher data types
    parquet_to_cypher_type_mapping = {
        "string": "STRING",
        "int32": "INTEGER",
        "int64": "INTEGER",
        "number": "FLOAT",
        "float": "FLOAT",
        "double": "FLOAT",
        "boolean": "BOOLEAN",
        "object": "MAP",
        "array": "LIST",
        "list<element: string>": "INT64[]",
        "null": "NULL",
        "date": "DATE",
        "time": "TIME",
        "datetime": "DATETIME",
        "timestamp": "DATETIME",
        "any": "ANY",
    }

    # Generate Cypher field type statements
    cypher_fields_from_parquet_schema = ", ".join(
        [
            # note: we use string splitting here for nested types
            # for ex. list<element: string>
            f"{field.name} {parquet_to_cypher_type_mapping.get(str(field.type))}"
            for field in parquet_schema
        ]
    )

    # branch for creating node table
    if table_type == "node":
        return (
            f"CREATE NODE TABLE {table_name}"
            f"({cypher_fields_from_parquet_schema}, "
            f"PRIMARY KEY ({table_pkey_parquet_field_name}))"
        )

    # else we return for rel tables
    return (
        f"CREATE REL TABLE {table_name}"
        f"(FROM {rel_table_field_mapping['from']} TO {rel_table_field_mapping['to']}, "
        f"{cypher_fields_from_parquet_schema})"
    )

In [114]:
for path in pathlib.Path(parquet_dir).glob("*"):

    first_pq_file = next(pathlib.Path(path).glob("*.parquet"))

    if first_pq_file.parent.name == "nodes":
        kz_conn.execute(
            generate_cypher_table_create_stmt_from_parquet_file(
                parquet_file=first_pq_file,
                table_type="node",
                table_name="Nodes",
                table_pkey_parquet_field_name="id",
            )
        )
    elif first_pq_file.parent.name == "edges":
        kz_conn.execute(
            generate_cypher_table_create_stmt_from_parquet_file(
                parquet_file=first_pq_file,
                table_type="rel",
                table_name="Rel",
                table_pkey_parquet_field_name="id",
            )
        )